<a href="https://colab.research.google.com/github/JerryChenz/InvestmentManagement/blob/master/stock_screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: Set inputs

We load the sample dataset from the my github repository, and display the dataset in pandas.

In [22]:
import pandas as pd
summary_url = 'https://raw.githubusercontent.com/JerryChenz/InvestmentManagementOpen/main/financial_models/Opportunities/Screener/screener_summary.csv'
df = pd.read_csv(summary_url)
df = df
df = df.fillna(0)
display(df)

,Ticker,Name,Exchange,Price,Price_currency,Shares,Reporting_Currency,Fx_rate,Dividend,Buyback,...,EBIT,EbitMargin,Avg_ebit_margin,Avg_ebit_growth,InterestExpense,NetIncomeCommonStockholders,NetMargin,Avg_net_margin,Avg_NetIncome_growth,Years_of_data
0,0001.HK,CKH HOLDINGS,HKG,48.80,HKD,3.830040e+09,HKD,1.000000,2.513551,0.323495,...,1.195010e+11,42.55,44.226667,-6.75,1.021400e+10,3.348400e+10,11.92,12.060000,-5.97,3.0
1,0002.HK,CLP HOLDINGS,HKG,57.40,HKD,2.526450e+09,HKD,1.000000,3.100002,0.000000,...,2.662600e+10,31.71,32.693333,0.02,1.383000e+09,8.491000e+09,10.11,9.976667,60.06,3.0
2,0003.HK,HK & CHINA GAS,HKG,7.65,HKD,1.865990e+10,HKD,1.000000,0.000000,0.000000,...,2.097190e+10,39.15,43.920000,5.67,1.408200e+09,5.017000e+09,9.37,13.733333,-15.12,3.0
3,0004.HK,WHARF HOLDINGS,HKG,22.50,HKD,3.056030e+09,HKD,1.000000,0.000000,0.000000,...,9.290000e+09,41.51,49.563333,8.19,3.160000e+08,6.019000e+09,26.90,21.790000,34.94,3.0
4,0005.HK,HSBC HOLDINGS,HKG,53.15,HKD,1.996830e+10,USD,7.810095,0.319657,0.277590,...,5.205200e+10,81.79,81.533333,-4.34,9.699000e+09,1.260700e+10,19.81,11.476667,94.36,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1139,9993.HK,RADIANCE HLDGS,HKG,3.97,HKD,4.045230e+09,CNY,1.138261,0.000000,0.000000,...,5.102718e+09,12.75,15.216667,6.53,3.498850e+08,3.274774e+09,8.18,8.936667,14.70,3.0
1140,9995.HK,REMEGEN-B,HKG,60.95,HKD,4.898370e+08,CNY,1.138261,0.000000,0.000000,...,8.739320e+08,61.38,0.000000,81.27,5.323000e+06,2.762580e+08,19.40,0.000000,-38.71,3.0
1141,9996.HK,PEIJIA-B,HKG,10.46,HKD,6.718290e+08,CNY,1.138261,0.000000,0.000000,...,-1.120230e+08,-82.05,-195.423333,94.44,1.860000e+05,-5.742160e+08,-420.57,-2872.370000,108.31,3.0
1142,9997.HK,KANGJI MEDICAL,HKG,8.76,HKD,1.218390e+09,CNY,1.138261,0.036808,0.041919,...,4.537370e+08,65.73,66.533333,15.18,1.960000e+05,4.567890e+08,66.18,52.036667,51.87,3.0


In [35]:
# capitalization in reporting currency
capitalization_price = df['Price'] * df['Shares']
capitalization_report = capitalization_price * df['Fx_rate']
total_debt = df['CurrentDebtAndCapitalLeaseObligation'] + df['LongTermDebtAndCapitalLeaseObligation']
# more easily realizable non-operating assets
monetary_assets = df['OtherShortTermInvestments'] + df['InvestmentinFinancialAssets']
# less easily realizable non-operating assets
fixed_nonop_assets = (df['InvestmentProperties'] + df['LongTermEquityInvestment']) * 0.5
total_nonop_assets = monetary_assets + fixed_nonop_assets
enterprise_value = capitalization_report + total_debt + df['MinorityInterest'] - df['CashAndCashEquivalents'] - total_nonop_assets
# liquidity_coverage_ratio
lcr = monetary_assets / df['CurrentLiabilities']
current_ratio = df['CurrentAssets'] / df['CurrentLiabilities']
# dividend rate & buyback rate
dividend_rate = df['Dividend'] / df['Price']
buyback_rate = df['Buyback'] / df['Price']
# Net PPE/ Sales
ppe_multiple = df['NetPPE'] / df['TotalRevenue']

# Step 2: Screening Criteria

We can screen using different sets of conditions, then merge them later.

In [39]:
# common fitlering conditions
common_1 = df['CashAndCashEquivalents'] > df['CurrentDebtAndCapitalLeaseObligation']
common_2 = lcr >= 1.2
common_3 = (total_debt / df['CurrentAssets']) < 1.1
common_4 = df['Avg_Gross_margin'] > 20

In [43]:
# 1st set of conditions
condition_1 = capitalization_price > 10000000000
condition_2 = df['Avg_ebit_margin'] > 20

In [41]:
# 2nd set of conditions
condition_5 = capitalization_price <= 10000000000
condition_6 = dividend_rate > 0.01
condition_7 = buyback_rate > 0.01

#Step 3. Output

Filter the dataset using the above conditions

In [42]:
# filtered by common conditions
common_df = df
common_df['EV'] = enterprise_value
common_df['Dividend rate'] = dividend_rate
common_df = common_df.loc[common_1 & common_2 & common_3 & common_4]
print(common_df.to_string())

       Ticker             Name Exchange    Price Price_currency        Shares Reporting_Currency   Fx_rate  Dividend   Buyback       Last_fy   TotalAssets  CurrentAssets  CurrentLiabilities  CurrentDebtAndCapitalLeaseObligation  CurrentCapitalLeaseObligation  LongTermDebtAndCapitalLeaseObligation  LongTermCapitalLeaseObligation  TotalEquityGrossMinorityInterest  MinorityInterest  CashAndCashEquivalents  OtherShortTermInvestments  InvestmentProperties  LongTermEquityInvestment  InvestmentinFinancialAssets        NetPPE  TotalRevenue  Avg_sales_growth  CostOfRevenue  GrossMargin  Avg_Gross_margin  SellingGeneralAndAdministration          EBIT  EbitMargin  Avg_ebit_margin  Avg_ebit_growth  InterestExpense  NetIncomeCommonStockholders  NetMargin  Avg_net_margin  Avg_NetIncome_growth  Years_of_data            EV  Dividend rate
16    0020.HK      SENSETIME-W      HKG    2.200            HKD  2.594880e+10                CNY  1.138261  0.000000  0.000000  1.640910e+12  3.666666e+10   2.139821e

In [44]:
# fitlered by 1st set of conditions
df_1 = common_df
df_1 = df_1.loc[condition_1 & condition_2]
print(df_1.to_string())

       Ticker             Name Exchange   Price Price_currency        Shares Reporting_Currency   Fx_rate  Dividend   Buyback       Last_fy   TotalAssets  CurrentAssets  CurrentLiabilities  CurrentDebtAndCapitalLeaseObligation  CurrentCapitalLeaseObligation  LongTermDebtAndCapitalLeaseObligation  LongTermCapitalLeaseObligation  TotalEquityGrossMinorityInterest  MinorityInterest  CashAndCashEquivalents  OtherShortTermInvestments  InvestmentProperties  LongTermEquityInvestment  InvestmentinFinancialAssets        NetPPE  TotalRevenue  Avg_sales_growth  CostOfRevenue  GrossMargin  Avg_Gross_margin  SellingGeneralAndAdministration          EBIT  EbitMargin  Avg_ebit_margin  Avg_ebit_growth  InterestExpense  NetIncomeCommonStockholders  NetMargin  Avg_net_margin  Avg_NetIncome_growth  Years_of_data            EV  Dividend rate
43    0083.HK        SINO LAND      HKG   10.22            HKD  8.061260e+09                HKD  1.000000  0.378810  0.000000  1.656550e+12  1.789840e+11   7.142499e+1

In [45]:
# filtered by 2nd set of conditions
df_2 = common_df
df_2 = df_2.loc[condition_5 & (condition_6 | condition_7)]
print(df_2.to_string())

       Ticker             Name Exchange   Price Price_currency        Shares Reporting_Currency   Fx_rate  Dividend   Buyback       Last_fy   TotalAssets  CurrentAssets  CurrentLiabilities  CurrentDebtAndCapitalLeaseObligation  CurrentCapitalLeaseObligation  LongTermDebtAndCapitalLeaseObligation  LongTermCapitalLeaseObligation  TotalEquityGrossMinorityInterest  MinorityInterest  CashAndCashEquivalents  OtherShortTermInvestments  InvestmentProperties  LongTermEquityInvestment  InvestmentinFinancialAssets        NetPPE  TotalRevenue  Avg_sales_growth  CostOfRevenue  GrossMargin  Avg_Gross_margin  SellingGeneralAndAdministration          EBIT  EbitMargin  Avg_ebit_margin  Avg_ebit_growth  InterestExpense  NetIncomeCommonStockholders  NetMargin  Avg_net_margin  Avg_NetIncome_growth  Years_of_data            EV  Dividend rate
24    0032.HK  CROSS-HAR(HOLD)      HKG  10.800            HKD  3.726880e+08                HKD  1.000000  0.420510  0.000000  1.640910e+12  8.021072e+09   3.960174e+0

In [46]:
# combine the results
result_set = pd.concat([df_1, df_2])
result_set = result_set.groupby("Ticker").first()
result_set = result_set.sort_values(by=['EV', 'Dividend rate'], ascending=[True, False]).reset_index()
print(result_set.to_string())

     Ticker             Name Exchange    Price Price_currency        Shares Reporting_Currency   Fx_rate  Dividend   Buyback       Last_fy   TotalAssets  CurrentAssets  CurrentLiabilities  CurrentDebtAndCapitalLeaseObligation  CurrentCapitalLeaseObligation  LongTermDebtAndCapitalLeaseObligation  LongTermCapitalLeaseObligation  TotalEquityGrossMinorityInterest  MinorityInterest  CashAndCashEquivalents  OtherShortTermInvestments  InvestmentProperties  LongTermEquityInvestment  InvestmentinFinancialAssets        NetPPE  TotalRevenue  Avg_sales_growth  CostOfRevenue  GrossMargin  Avg_Gross_margin  SellingGeneralAndAdministration          EBIT  EbitMargin  Avg_ebit_margin  Avg_ebit_growth  InterestExpense  NetIncomeCommonStockholders  NetMargin  Avg_net_margin  Avg_NetIncome_growth  Years_of_data            EV  Dividend rate
0   3983.HK   CHINA BLUECHEM      HKG    1.900            HKD  1.796000e+09                CNY  1.138261  0.205345  0.000000  1.640910e+12  2.203146e+10   1.394292e+10 